2019.12.31 기준
각 지역(25개지역구)별 상표 순위

금천구 대상으로 각 주유소 1년치 휘발유 가격, 국제 유가(WTI, 브렌트, 두바이유가)의 상관계수

In [28]:
import os
import zipfile
import pandas as pd
import numpy as np

In [4]:
path = './data/'
os.listdir(path)

['주유소-20250618T053441Z-1-001.zip',
 '주유소-20250618T053441Z-1-001.zip:Zone.Identifier']

In [8]:
for file in os.listdir(path):
    if file.split('.')[-1]=='zip':
        with zipfile.ZipFile(path + file) as f:
            f.extractall(path)

In [29]:
path2 = './data/주유소/'
df = pd.concat( [ pd.read_csv(path2+file, encoding='euc-kr') for file in os.listdir(path2) if file.split('.')[-1] =='csv'] )

In [25]:
df.head(1)

,번호,지역,상호,주소,기간,상표,셀프여부,고급휘발유,휘발유,경유,실내등유
0,A0006039,서울 강남구,(주)동하힐탑셀프주유소,서울 강남구 논현로 640,20190101,SK에너지,셀프,1673,1465,1365,0


In [14]:
df.columns

Index(['번호', '지역', '상호', '주소', '기간', '상표', '셀프여부', '고급휘발유', '휘발유', '경유',
       '실내등유'],
      dtype='object')

In [93]:
rank = df.groupby(['지역', '상표']).count()[['번호']].sort_values(by=['지역', '번호'], ascending=False)
rank

번호
지역     상표          
서울 중랑구 S-OIL   1842
       GS칼텍스   1705
       SK에너지   1672
       현대오일뱅크   365
       알뜰주유소    135
...             ...
서울 강동구 S-OIL    730
서울 강남구 SK에너지   6540
       GS칼텍스   4042
       S-OIL   2190
       현대오일뱅크  1825

[114 rows x 1 columns]

In [39]:
gs = df[df['지역'].str.find('금천구') > -1]['상호'].unique()

,상호,기간
29155,(주)진우에너지 백운주유소,20190101
29156,(주)진우에너지 백운주유소,20190102
29157,(주)진우에너지 백운주유소,20190103
29158,(주)진우에너지 백운주유소,20190104
29159,(주)진우에너지 백운주유소,20190105
...,...,...
29190,(주)진우에너지 백운주유소,20191227
29191,(주)진우에너지 백운주유소,20191228
29192,(주)진우에너지 백운주유소,20191229
29193,(주)진우에너지 백운주유소,20191230


In [33]:
from bs4 import BeautifulSoup as bs
import requests
import numpy as np

np_date = np.array([])
np_p  = np.array([])

url_wti = 'https://finance.naver.com/marketindex/worldDailyQuote.naver?fdtc=2&marketindexCd=OIL_CL&page={}'
n = 1
while True:
    soup = bs(requests.get(url_wti.format(n)).content).find('tbody').find_all('tr')
    for tr in soup:
        date = tr.find('td', class_='date').text.replace("\t", "").replace("\n", "").replace(".", "")
        p = tr.find('td', class_='num').text.replace("\t", "").replace("\n", "")

        np_date = np.append(np_date, date)   
        np_p = np.append(np_p, p)
    n +=1
    if int(date) < 20180101:
        break

In [36]:
np_wti = np.vstack([np_date, np_p])

In [37]:
np_wti

array([['20250617', '20250616', '20250613', ..., '20171227', '20171226',
        '20171222'],
       ['73.27', '70.25', '72.98', ..., '59.64', '59.97', '58.47']],
      shape=(2, 1883), dtype='<U32')

In [64]:
df2 = pd.DataFrame(np_wti.transpose(), columns=['기간', 'wti가격'])
df2['기간'] = df2['기간'].astype('int64')
corrs_wti = []
for gs_name in gs:
    tmp = pd.merge(df[df['상호'] == gs_name][['상호', '기간','고급휘발유','휘발유']],  df2, on='기간', how='left')
    corrs_wti.append(tmp[['휘발유', 'wti가격']].dropna().corr().iloc[0, 1])

In [66]:
tmp

,상호,기간,고급휘발유,휘발유,wti가격
0,지에스칼텍스(주)일신주유소,20190101,1566,1346,NaN
1,지에스칼텍스(주)일신주유소,20190102,1566,1346,46.54
2,지에스칼텍스(주)일신주유소,20190103,1566,1346,47.09
3,지에스칼텍스(주)일신주유소,20190104,1566,1346,47.96
4,지에스칼텍스(주)일신주유소,20190105,1566,1346,NaN
...,...,...,...,...,...
359,지에스칼텍스(주)일신주유소,20191227,1708,1558,61.72
360,지에스칼텍스(주)일신주유소,20191228,1708,1558,NaN
361,지에스칼텍스(주)일신주유소,20191229,1708,1558,NaN
362,지에스칼텍스(주)일신주유소,20191230,1708,1558,61.68


In [67]:
tmp[['휘발유', 'wti가격']].dropna().corr()

,휘발유,wti가격
휘발유,1.00000,0.19394
wti가격,0.19394,1.00000


In [65]:
corrs_wti

[np.float64(0.22172367083403166),
 np.float64(0.20220910131303768),
 np.float64(0.20050878076105558),
 np.float64(-0.04085420299626117),
 np.float64(-0.032853094436273766),
 np.float64(0.21056388955382285),
 np.float64(0.20213619361521643),
 np.float64(0.2092479467050484),
 np.float64(0.21047482948671323),
 np.float64(0.20054178461885255),
 np.float64(0.23632040802811183),
 np.float64(0.17570509400946582),
 np.float64(0.19394021271984446)]

In [60]:
df2 = pd.DataFrame(np_wti.transpose(), columns=['기간', 'wti가격'])
df2['기간'] = df2['기간'].astype('int64')
pd.merge(df[df['상호'] == gs[0]][['상호', '기간','고급휘발유','휘발유']],  df2, on='기간', how='left')

,상호,기간,고급휘발유,휘발유,wti가격
0,(주)진우에너지 백운주유소,20190101,0,1365,NaN
1,(주)진우에너지 백운주유소,20190102,0,1345,46.54
2,(주)진우에너지 백운주유소,20190103,0,1345,47.09
3,(주)진우에너지 백운주유소,20190104,0,1345,47.96
4,(주)진우에너지 백운주유소,20190105,0,1345,NaN
...,...,...,...,...,...
360,(주)진우에너지 백운주유소,20191227,0,1545,61.72
361,(주)진우에너지 백운주유소,20191228,0,1545,NaN
362,(주)진우에너지 백운주유소,20191229,0,1545,NaN
363,(주)진우에너지 백운주유소,20191230,0,1545,61.68


0       20250617
1       20250616
2       20250613
3       20250612
4       20250611
          ...   
1878    20171229
1879    20171228
1880    20171227
1881    20171226
1882    20171222
Name: 기간, Length: 1883, dtype: int64

In [47]:
pd.DataFrame(np_wti.transpose()).shape

(1883, 2)

In [48]:
df[df['상호'] == gs[0]][['상호', '기간']].shape

(365, 2)

In [56]:
pd.DataFrame(np_wti.transpose()).iloc[:,0].unique().shape

(1883,)